In [57]:
import spacy
from datetime import datetime, timedelta


In [58]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import os

In [59]:
def get_google_calendar_service():
    # Set up the Google Calendar API credentials (you need to replace 'token.json' with your token file)
    creds = None
    SCOPES = ['https://www.googleapis.com/auth/calendar']

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    # Build and return the Google Calendar API service
    service = googleapiclient.discovery.build('calendar', 'v3', credentials=creds)
    return service

get_google_calendar_service()

In [60]:
def get_events():
    # Call the Calendar API
    service = get_google_calendar_service()
    now = datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    events_result = service.events().list(calendarId='primary', timeMin=now,
                                          maxResults=10, singleEvents=True,
                                          orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(start, event['summary'])
get_events()

2024-03-11T09:00:00-04:00 EECS 545
2024-03-11T14:30:00-04:00 STATS 426
2024-03-12T16:00:00-04:00 SI 649
2024-03-13T09:00:00-04:00 EECS 545
2024-03-13T14:30:00-04:00 STATS 426
2024-03-13T16:00:00-04:00 SI 649 - DIS
2024-03-14T12:00:00-04:00 Company Day
2024-03-14T13:00:00-04:00 Dwaine Campbell
2024-03-18T09:00:00-04:00 EECS 545
2024-03-18T14:30:00-04:00 STATS 426


In [63]:
def moveEvent(event_details):
    event_title = event_details.get('name', '')
    date = event_details.get('date', '')
    time = event_details.get('time', '')
    new_date = event_details.get('new_date', '')
    new_time = event_details.get('new_time', '')
    service = get_google_calendar_service()

    print(event_title, date, time)

    if new_date and new_time:
        new_datetime = datetime.strptime(f'{new_date} {new_time}', '%Y-%m-%d %I:%M %p')

    elif new_time:
        new_datetime = datetime.now().replace(hour=int(time.split(':')[0]), minute=int(time.split(':')[1]))
    else:
        #TODO: add subfunction to ask LLM for new date or time
        print('Please provide a new date or time to move the event to.') 
        return
    
    if date and time:
        current_datetime = datetime.strptime(f'{date} {time}', '%Y-%m-%d %I:%M %p')
    elif time:
        current_datetime = datetime.now().replace(hour=int(time.split(':')[0]), minute=int(time.split(':')[1]))
    elif date:
        current_datetime = datetime.strptime(date, '%Y-%m-%d').replace(hour=0, minute=0, second=0)

    if not date and not time:   
        now = datetime.utcnow().isoformat() + 'Z'
        events_result = service.events().list(calendarId='primary', timeMin=now, orderBy='startTime', singleEvents=True, q=f'{event_title.lower()}').execute() #matches with mispelled event names or lower case
        events = events_result.get('items', [])
        if events:
            event_to_move = events[0]
        else:
            print(f'Event "{event_title}" not found.')
            return
    else:
        events_result = service.events().list(calendarId='primary', timeMin=current_datetime.isoformat()+'Z', orderBy='startTime', singleEvents=True, q=f'{event_title.lower()}').execute() #matches with mispelled event names or lower case
        events = events_result.get('items', [])
        if events:
            event_to_move = events[0]
        else:
            print(f'Event "{event_title}" not found.')
            return

    print(event_to_move)
    start_datetime = datetime.fromisoformat(event_to_move['start']['dateTime'])  
    end_datetime = datetime.fromisoformat(event_to_move['end']['dateTime'])
    event_duration = end_datetime - start_datetime
    new_datetime_end = new_datetime + event_duration
    event_to_move['start']['dateTime'] = new_datetime.isoformat()
    event_to_move['end']['dateTime'] = new_datetime_end.isoformat()

    # Update the event
    updated_event = service.events().update(
    calendarId='primary', eventId=event_to_move['id'], body=event_to_move).execute()

    print(f'Event "{event_title}" moved to {updated_event["start"]["dateTime"]}')

   
    #expand scope: queries like move all my meetings on saturday to monday
    

In [66]:
user_entities = {'name':'EECS', 'date':'2024-03-13', 'new_date':'2024-03-12', 'new_time':'11:00 AM'}
moveEvent(user_entities) 

#issue: doesn't work when name partially matches 

EECS 2024-03-13 
{'kind': 'calendar#event', 'etag': '"3409717985262000"', 'id': 'cdi3iophc8p38bb471i3cb9k6dgjab9occq36b9m60q3idr36dj64p1m6s_20240313T130000Z', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=Y2RpM2lvcGhjOHAzOGJiNDcxaTNjYjlrNmRnamFiOW9jY3EzNmI5bTYwcTNpZHIzNmRqNjRwMW02c18yMDI0MDMxM1QxMzAwMDBaIHNocml5YXNlQHVtaWNoLmVkdQ', 'created': '2024-01-10T03:56:11.000Z', 'updated': '2024-01-10T03:56:32.631Z', 'summary': 'EECS 545', 'location': 'STAMPS', 'colorId': '1', 'creator': {'email': 'shriyase@umich.edu', 'self': True}, 'organizer': {'email': 'shriyase@umich.edu', 'self': True}, 'start': {'dateTime': '2024-03-13T09:00:00-04:00', 'timeZone': 'America/New_York'}, 'end': {'dateTime': '2024-03-13T10:30:00-04:00', 'timeZone': 'America/New_York'}, 'recurringEventId': 'cdi3iophc8p38bb471i3cb9k6dgjab9occq36b9m60q3idr36dj64p1m6s', 'originalStartTime': {'dateTime': '2024-03-13T09:00:00-04:00', 'timeZone': 'America/New_York'}, 'iCalUID': 'cdi3iophc8p38bb471i3c